# Ping Modeller
### Ewan Miles - 06/05/2020

**This code is entirely open-source and thus editable by any user.**

This script is essentially a simple make-up of two functions: **ping()** and **pingmodel()**, with the latter requiring the former.

**ping()** uses subprocess to ping a user-input host address (typically a URL, e.g. "google.com", entered as a string). This means it sends a single arbitrary packet to the address and measures the time from departure to return, as a representation of connection strength. The data returned includes alot of information, but for the purposes of this script, **ping()** stringifies it and slices to the ping time only, returning it as a float.

**pingmodel()** uses **ping()** repeatedly over a user-defined period, appending arrays with ping values in conjunction with the time at which the ping was sent, plotting the variation in ping (ms) across a time interval (s). Due to the structure of the function, _the graph does not plot until the time has passed_. It also features a break in the ping loop if packets do not return; it does this by checking if the ping array is filled with _None_ values only. A packet will not return if the connection is very slow, but likelihood is that, if no packets return over a few seconds, the host is not reachable (i.e. the user has likely input an incorrect URL).

After running the module imports and defined functions, all that you need to do is run the last cell with **pingmodel()**, where the two arguments are the host name and the time interval over which to ping, respectively, e.g. `pingmodel("google.com",30)`, which would ping google.com over 30 seconds and plot the graph.

* Great connections have ping averages in the range of 10-20ms, with low standard deviations.
* Good connections have ping averages in the range of 40-60ms, with low to medium standard deviations (medium deviations usually caused by occasional "spikes" in ping).
* Poor connections typically average over 150-200ms, or have high standard deviations, or both.
* **Virtually non-existent connections typically average 1000ms+.** I have been unfortunate enough to be on these connections many a time.

To give an idea of what the ping represents, playing a game on 1000ms ping means that any controls by the player will be registered in-game after (around) a one second delay.

In [ ]:
### IMPORTING MODULES ###

%matplotlib notebook

import subprocess
import matplotlib.pyplot as plt
from time import time, sleep
import numpy as np

In [ ]:
def ping(host):
    """
    Function which uses subprocess module to ping host website of choice, input:
    - host: URL of website to ping, e.g. "youtube.com"
    Outputs ping to website from computer in ms, one count
    """

    # Ping the host website, output "out" variable as byte response for packet return
    ping = subprocess.Popen(
        ["ping", "-c", "1", host],
        stdout = subprocess.PIPE,
    )
    out = ping.communicate()
    
    # Try to stringify the ping output and find and slice the return time (ms)
    try:
        out = str(out)
        startms = out.find("time=")
        endms = out.find("ms")
    
        time = float(out[(startms + 5):(endms-1)])
        
    # Except lack of packet return, set time (ms) to None
    except ValueError:
        time = None

    return time

In [ ]:
def pingmodel(host, length):
    """
    Function which pings host website of choice over certain length of time, inputs:
    - host: URL of website to ping, e.g. "youtube.com"
    - length: length of time over which to ping IN SECONDS, e.g. 300 secs
    Outputs plot of ping(ms) vs. time(s) with average and st.dev
    NOTE the function sleeps for 0.05s between pings
    GRAPH WILL PRINT AFTER THE TIME HAS PASSED, E.G. IF length = 30, GRAPH WILL APPEAR AFTER 30s
    """
    
    # Take time at beginning of execution, to have a start point to check length of time passed
    start_time = current_time = time()
    
    # Empty arrays to fill with ping over time
    pingarray = []
    timearray = []
    
    # "broken" variable checks if packets have not returned, True in if statements below
    broken1 = broken2 = False
    
    while (current_time - start_time) < length:
        
        # Current time value of pinging host (x value), ping in ms (y value)
        ping_time = time() - start_time
        value = ping(host)
        
        # Append to lists
        timearray.append(ping_time)
        pingarray.append(value)
        
        # Check time to see if length has been met
        current_time = time()
        
        # Statements to break ping loop if packets do not return (no connection)
        if length/4 < 10:
            if (ping_time > length/4) and (set(pingarray) == {None}):
                print("{0}s have passed and no packets have returned yet, so the connection has been broken.".format(length/4))
                print("Are you sure '{0}' is the correct host URL? Perhaps it is not possible to ping this address.".format(host))
                broken1 = True
                break
        else:
            if (ping_time > 10) and (set(pingarray) == {None}):
                print("10s have passed and no packets have returned yet, so the connection has been broken.")
                print("Are you sure '{0}' is the correct host URL? Perhaps it is not possible to ping this address.".format(host))
                broken2 = True
                break
        
    # Plot data
    plt.figure()
    plt.xlabel("Time (s)")
    plt.ylabel("Ping (ms)")
    plt.grid()
    plt.plot(timearray,pingarray,"k-",label="Packet Return Time")
    
    if broken1 == True:
        plt.title("Pinging host '{0}',\nBroken after {1}s".format(host,length/4))

    elif broken2 == True:
        plt.title("Pinging host '{0}',\nBroken after 10s".format(host,length))
    
    else: 
        plt.title("Pinging host '{0}',\nlength approx. {1}s".format(host,length))
        
        pingarray = [i for i in pingarray if i != None]    # Remove None values to take mean and standard deviation
    
        # Calculate and plot mean, standard deviation
        x0 = np.mean(pingarray)
        sigma = np.std(pingarray)
    
        plt.axhline(y=x0,c="b",label="Mean, $x_0$")
        plt.axhline(y=x0+sigma,c="r",ls="-.",label="± St. Dev, $1\sigma$")
    
        # Only plot mean - sigma if above 0ms, otherwise raises graph pointlessly
        if (x0-sigma) > 0:
            plt.axhline(y=x0-sigma,c="r",ls="-.")
    
        print("\nAverage ping time: {0:0.2f}ms".format(x0))
        print("Standard Deviation: {0:0.2f}ms".format(sigma))
    
    plt.legend(loc="best") 

In [ ]:
### RUN THIS CELL TO PRODUCE A GRAPH ###

# Graph will not appear until time has passed
pingmodel("", )